# Analyse NVIDIA

Notebook recréé après corruption. Complétez librement vos analyses ci-dessous.

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")
%matplotlib inline

In [ ]:
ticker = "NVDA"
data = yf.download(ticker, period="2y")
data.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
data["Close"].plot(ax=ax, title=f"Cours de {ticker}")
plt.show()